In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import datetime
import time
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import random
import os
from urllib.parse import urlparse
import ssl
import pyautogui
ssl._create_default_https_context = ssl._create_unverified_context
pyautogui.FAILSAFE = False
from selenium.common.exceptions import NoSuchElementException
import re


counter = 0

In [12]:
# Function to initialize the Selenium WebDriver
def init_driver():
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Set the user data directory to store the session
    options.add_argument(r'--user-data-dir=C:\\Users\\Mahnoor-Zubair\\Desktop\\sessions')
    # options.add_argument('--headless')  # Add this for headless mode
    driver = uc.Chrome(options=options)

    # driver = uc.Chrome(version_main=133, options=options)
    return driver

In [13]:

def human_like_typing(element, text):
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(0.1, 0.3))  # Random delay between keystrokes

In [14]:
def send_message(driver, message):
    """
    Clicks the 'Message' button, checks if the message was already sent, and sends a new one only if necessary.
    
    :param driver: Selenium WebDriver instance
    :param message: Message to send
    """
    global wrote_message
    try:
        print("Trying to locate the 'Message' button...")
        message_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Message']"))
        )
        # move_cursor_to_element(message_button)
        print("Cursor moved to the element (pyautogui)")
        
        time.sleep(2)
        print("Message button found, clicking...")
        message_button.click()
    except Exception as e:
        print(f"Error clicking the Message button: {e}")
        return

    # Wait for chat to load
    time.sleep(5)

    try:
        print("Fetching previously sent messages...")
        messages = driver.find_elements(By.XPATH, "//div[contains(@class, 'html-div xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd xeuugli x1vjfegm')]//span")

        # Extract text and clean it up
        sent_messages = [msg.text.strip() for msg in messages if msg.text.strip()]
        print("Retrieved Messages:", sent_messages)  # Debugging

    except Exception as e:
        print(f"Error fetching messages: {e}")
        return

    # Check if the exact message is already sent
    if message.strip() in sent_messages:
        wrote_message=False
        print("Message already sent. Skipping...")
        
        return wrote_message

    try:
        print("Locating the message input box...")
        message_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Message' and @role='textbox']"))
        )
        # move_cursor_to_element(message_input)
        print("Cursor moved to the element (pyautogui)")
       
        print("Found message input box, typing message...")
        human_like_typing(message_input, message)
        time.sleep(random.uniform(0.5, 1.5))  # Small delay before hitting Enter
        message_input.send_keys(Keys.RETURN)
        time.sleep(random.uniform(5,6))  # Small delay before hitting Enter
        # actions = ActionChains(driver)
        # actions.move_to_element(message_input).click().send_keys(message.strip() + Keys.RETURN).perform()
        try:
            sent_element = driver.find_element(By.XPATH, "//span[contains(text(), 'Sent')]")
            print("Message sent successfully!")
            try:
                blocked_element = driver.find_element(By.XPATH, "//div[contains(text(), \"This account can't receive your message\")]")
                print('This id has blocked incoming mesages from a stranger')
                wrote_message = False
                return wrote_message
            except:
                print("Id not blocked the incoming messages from stranger")
                wrote_message = True
        except:
            print("The message to this profile couldnt send")
            wrote_message = False
    except Exception as e:
        print(f"Error typing message: {e}")
    return wrote_message


In [ ]:
def log_out(driver):
    global counter
    counter = 0
    try:
        # Wait for the profile button to be visible and clickable
        profile_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Your profile"]'))
        )
        profile_button.click()  # Click the profile button
        time.sleep(2)  # Slight wait after clicking the profile button

    except Exception as e:
        print("Profile button not found or clickable:", e)
    
    try:
        # Wait for the log-out element to be clickable
        log_out_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body//span[contains(text(), 'Log Out')]"))
        )
    except Exception as e:
        print("Log out element not found or clickable:", e)
    print(log_out_element.text)
    log_out_element.click()  # Click the Log Out button
    time.sleep(2)  # Slight wait after logging out

In [16]:
# Function to handle the login process
def main_handler(username, password, univer, index, requester):
    driver = init_driver()
    print("Now open the browser to login")
    driver.get("https://www.facebook.com")
    time.sleep(2)  # Allow page to load
    
    try:
        # Locate and enter username
        email_input = driver.find_element(By.ID, "email")
        time.sleep(3)
        email_input.send_keys(Keys.CONTROL + "a")  # Select all text
        email_input.send_keys(Keys.BACKSPACE)  # Delete the selected text
        time.sleep(3)
        email_input.send_keys(username)
        time.sleep(3)

        # Locate and enter password
        password_input = driver.find_element(By.ID, "pass")
        time.sleep(3)
        password_input.clear()
        time.sleep(3)
        password_input.send_keys(password)
        time.sleep(3)
        password_input.send_keys(Keys.RETURN)
        time.sleep(5)  # Wait for login to complete
        
    except:
        pass

    # Check login success
    if "login_attempt" in driver.current_url:
        print(f"Login failed for {username}")
    else:
        print(f"Login successful for {username}")
        time.sleep(5)
        # If university data is needed, process it here
        # Ensure the background thread starts only once
        
        
        if not univer.empty:

            try:
                university_name = univer.iloc[index]['university']
                print(f"Processing university: {university_name} for {username}")
                time.sleep(4)
                global counter
                counter += 4
                print(username, " : ", counter)
            except Exception as e:
                print(f"❌ Error during university verification: {str(e)}")
        
            # driver.get("https://www.facebook.com/archie.vanaswegen.1")
            # wrote_message = send_message(driver, "Hello, Hope you are doing well. Do let me know if you need assistance with anything")
            # if wrote_message:
            #     print("sent the message")
            # else:
            #     print("the message was not sent")
    log_out(driver)
    driver.quit()

In [25]:
if __name__ == "__main__":
    # Load account credentials
    accounts = pd.read_csv(r"C:\Users\Mahnoor-Zubair\Desktop\main - Copy\main - Copy\staticfiles\account.csv")
    univer = pd.read_csv(r"C:\Users\Mahnoor-Zubair\Desktop\main - Copy\main - Copy\staticfiles\uni.csv")

    
    for i, row in accounts.iterrows():
        
        uname = row['username']
        pas = row['password']
        main_handler(uname, pas, univer, i, "requester")


Now open the browser to login
Login successful for faizifaizan213faizi@gmail.com
Processing university: Abertay University for faizifaizan213faizi@gmail.com
faizifaizan213faizi@gmail.com  :  4
Logged out successfully


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x008BD363+60275]
	GetHandleVerifier [0x008BD3A4+60340]
	(No symbol) [0x006F06F3]
	(No symbol) [0x007020E1]
	(No symbol) [0x007011C0]
	(No symbol) [0x006F78F3]
	(No symbol) [0x006F5DFA]
	(No symbol) [0x006F913A]
	(No symbol) [0x006F91B7]
	(No symbol) [0x007333AE]
	(No symbol) [0x0075D08C]
	(No symbol) [0x0072DCF4]
	(No symbol) [0x0075D304]
	(No symbol) [0x0077E6EB]
	(No symbol) [0x0075CE86]
	(No symbol) [0x0072C623]
	(No symbol) [0x0072D474]
	GetHandleVerifier [0x00B08FE3+2467827]
	GetHandleVerifier [0x00B045E6+2448886]
	GetHandleVerifier [0x00B1F80C+2560028]
	GetHandleVerifier [0x008D3DF5+153093]
	GetHandleVerifier [0x008DA3BD+179149]
	GetHandleVerifier [0x008C4BB8+91080]
	G

In [ ]:
# import subprocess

# def get_wifi_password(profile_name):
#     try:
#         result = subprocess.check_output(
#             ['netsh', 'wlan', 'show', 'profile', profile_name, 'key=clear'],
#             text=True
#         )
#         for line in result.split('\n'):
#             if "Key Content" in line:
#                 return line.split(":")[1].strip()
#         return "Password not found or not stored."
#     except subprocess.CalledProcessError:
#         return "Profile not found."

# # Example usage:
# wifi_name = input("Enter the Wi-Fi name (SSID): ")
# password = get_wifi_password(wifi_name)
# print(f"Password for '{wifi_name}': {password}")


In [ ]:
saved_messages = [
    "Our working hours are 9am to 5pm, Monday to Friday.",
    "You can reset your password by clicking on 'Forgot Password' at login.",
    "We usually reply to queries within 24 hours.",
    "Shipping typically takes 3-5 business days.",
    "Yes, we offer a 30-day money-back guarantee.",
    "You can contact support via email or live chat.",
    "Please ensure your billing info is up to date.",
    "We accept all major credit cards and PayPal.",
    "Yes, we ship internationally.",
    "Thank you for reaching out! We'll get back to you shortly."
]


In [ ]:
# Install the required package
%pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode saved messages
saved_embeddings = model.encode(saved_messages, convert_to_tensor=True)



[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/345.7 kB ? eta -:--:--
   -- ------------------------------------- 20.5/345.7 kB ? eta -:--:--
   ----- ---------------------------------- 51.2/345.7 kB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 51.2/345.7 kB 1.3 MB/s eta 0:00:01
   -------- ------------------------------ 71.7/345.7 kB 491.5 kB/s eta 0:00:01
   ----------- -------------------------- 102.4/345.7 kB 535.8 kB/s eta 0:00:01
   ------------- ------------------------ 122.9/345.7 kB 554.9 kB/s eta 0:00:01
   ---------------- --------------------- 153.6/345.7 kB 573.4 kB/s eta 0:00:01
   ------------------ ------------------- 163.8/345.7 kB 517.2 kB/s eta 0:00:01
   ---------------------- --------------- 204.8/345.7 kB 565.6 kB/s eta 0:00:01
   ----------------------- -------------- 215.0/345.7 kB 546.1 kB/s eta 0:00:01
   --------------------------- ---------- 245.8/345.7 kB 537.7 kB/s eta 0:00:01
   -------------------------------- ----- 297.0/345.7 kB 61

c:\Users\Mahnoor-zubair\Desktop\main - Copy\main - Copy\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Mahnoor-zubair\Desktop\main - Copy\main - Copy\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahnoor-zubair\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate De

In [ ]:
def get_best_reply(user_input):
    # Convert user message to embedding
    query_embedding = model.encode(user_input, convert_to_tensor=True)
    
    # Compare it with the saved message embeddings
    scores = util.pytorch_cos_sim(query_embedding, saved_embeddings)
    
    # Get the index of the most similar response
    best_match_idx = scores.argmax().item()
    
    # Return the best-matching saved message
    return saved_messages[best_match_idx]


In [ ]:
# Try it
user_question = "how long does it take to ship?"
print("Bot:", get_best_reply(user_question))

Bot: Shipping typically takes 3-5 business days.
